### Import libraries and start Spark context

In [9]:
from operator import add
from pyspark import SparkConf, SparkContext
from time import time
conf = (SparkConf() 
.setMaster("spark://192.168.2.34:7077")
.setAppName("test_s3").set("spark.driver.host", "192.168.2.34"))
    
spark_context = SparkContext(conf = conf)

### Preprocess the data to get the list of all nucleotides

In [10]:
lines = spark_context.textFile("hdfs://192.168.2.34:9000/genomeData/chromosome1")
lines_without_headers = lines.filter(lambda line : ">" not in line)
list_of_letters = lines_without_headers.flatMap(lambda line: list(line))

### Preprocess data for counting

In [11]:
list_of_letters_with_values = list_of_letters.map(lambda letter: (letter,1))
list_of_letters_with_values.take(5)

[('N', 1), ('N', 1), ('N', 1), ('N', 1), ('N', 1)]

### Do the reduce operation 

In [12]:
results = list_of_letters_with_values.reduceByKey(add)
results.take(5)

[('N', 23970000),
 ('C', 47024412),
 ('T', 65668756),
 ('A', 65570891),
 ('G', 47016562)]

### Do all in one cell to measure time more easily

In [13]:
count = lines.count()

In [14]:
lines = spark_context.textFile("hdfs://192.168.2.34:9000/genomeData/chromosome1")
count = lines.count()
quarter_lines = lines.take(count//4)
quarter_lines = spark_context.parallelize(quarter_lines)
start = time()
results = quarter_lines\
    .filter(lambda line : ">" not in line)\
    .flatMap(lambda line: list(line))\
    .map(lambda letter: (letter,1))\
    .reduceByKey(add)
results.take(5)
time() - start

25.536808013916016

In [15]:
spark_context.stop()